In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score,confusion_matrix,accuracy_score
import xgboost as xgb

from pylab import rcParams
rcParams['figure.figsize']=8,8

In [2]:
data=pd.read_csv("creditcard.csv")

In [3]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
data.shape

(284807, 31)

In [5]:
data.Class.value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

In [6]:
X = data.iloc[:,:-1]
Y = data.iloc[:,-1]

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0, stratify = Y)

In [8]:
print("Xtrain shape:",X_train.shape)
print("Ytrain shape:",Y_train.shape)

Xtrain shape: (227845, 30)
Ytrain shape: (227845,)


In [10]:
dtrain=xgb.DMatrix(X_train,label=Y_train)
dtest=xgb.DMatrix(X_test)

c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use

In [12]:
parameters={
    'max_depth':7,
    'eta':1,
    'silent':1,
    'objective':'binary:logistic',
    'eval_metric':'auc',
    'learning_rate':0.05
}

In [13]:
num_round=50
from datetime import datetime
start = datetime.now()
xg=xgb.train(parameters,dtrain,num_round)
stop=datetime.now()

c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [15:07:07] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


In [14]:
train_class_pred_probs=xg.predict(dtrain)
test_class_pred_probs=xg.predict(dtest)

In [15]:
train_class_pred=[]
test_class_pred=[]
for i in range(0,len(train_class_pred_probs)):
    if train_class_pred_probs[i]>=0.5:
        train_class_pred.append(1)
    else:
        train_class_pred.append(0)

for i in range(0,len(test_class_pred_probs)):
    if test_class_pred_probs[i] >= 0.5:
        test_class_pred.append(1)
    else:
        test_class_pred.append(0)            

In [16]:
test_class_pred_probs[:20]

array([0.00995724, 0.00995724, 0.00995724, 0.00995724, 0.00995724,
       0.00995724, 0.01013789, 0.00995724, 0.00995724, 0.00995724,
       0.00995724, 0.00995724, 0.00995724, 0.00995724, 0.00995724,
       0.00995724, 0.00995724, 0.00995724, 0.00995724, 0.00995724],
      dtype=float32)

In [17]:
len(Y_train)

227845

In [18]:
len(train_class_pred)

227845

In [19]:
train_accuracy_xgb = accuracy_score(train_class_pred,Y_train)
test_accuracy_xgb = accuracy_score(test_class_pred,Y_test)

print("The accuracy on train data is ", train_accuracy_xgb)
print("The accuracy on test data is ", test_accuracy_xgb)

The accuracy on train data is  0.9997147183392219
The accuracy on test data is  0.999385555282469


In [20]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
test_accuracy_xgb = accuracy_score(test_class_pred,Y_test)
test_precision_xgb = precision_score(test_class_pred,Y_test)
test_recall_score_xgb = recall_score(test_class_pred,Y_test)
test_f1_score_xgb = f1_score(test_class_pred,Y_test)
test_roc_score_xgb = roc_auc_score(test_class_pred,Y_test)

print("The accuracy on test data is ", test_accuracy_xgb)
print("The precision on test data is ", test_precision_xgb)
print("The recall on test data is ", test_recall_score_xgb)
print("The f1 on test data is ", test_f1_score_xgb)
print("The roc_score on train data is ", test_roc_score_xgb)

The accuracy on test data is  0.999385555282469
The precision on test data is  0.7142857142857143
The recall on test data is  0.9090909090909091
The f1 on test data is  0.8
The roc_score on train data is  0.9542993439714896
